<a href="https://colab.research.google.com/github/DavideScassola/data_analysis_laboratory/blob/main/notebooks/Prova_Pratica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prova in classe

Ora tocca voi, sulla base del notebook visto in classe provate a creare un albero decisionale per classificare la specie dei pinguini



In [ ]:
# Importa le librerie necessarie
import pandas as pd

LINK_DATI = 'https://raw.githubusercontent.com/DavideScassola/data_analysys_laboratory/main/notebooks/dati/penguins.csv'
df = pd.read_csv(LINK_DATI)

# Visualizza le prime righe del DataFrame
df.head()



*   Prova a descrivere i dati, quali sono le variabili?
*   Cosa potremmo fare con i valori mancanti?


In [ ]:
# dimensioni dataset
df.shape

In [ ]:
# Visualizzare il numero di valori mancanti per ogni colonna
valori_mancanti_per_colonna = df.isna().sum()
print("Valori mancanti per colonna:\n", valori_mancanti_per_colonna)

# Eliminare le righe con valori mancanti
df_senza_mancanti = df.dropna()

# Visualizzare il DataFrame senza valori mancanti
print("\nDataFrame senza valori mancanti:\n", df_senza_mancanti)

# EDA

 L'analisi esplorativa dei dati (EDA) è un passo cruciale per comprendere meglio il tuo dataset. Puoi utilizzare diverse librerie Python per condurre l'EDA, come `matplotlib`, `seaborn`, e `pandas`.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Crea uno scatter plot per i valori continui (esempio: 'bill_length_mm' vs 'bill_depth_mm')
sns.scatterplot(x='bill_length_mm', y='bill_depth_mm', hue='species' , data=df_senza_mancanti)

# Aggiungi titolo e etichette degli assi
plt.title('Scatter Plot - Bill Length vs Bill Depth')
plt.xlabel('Bill Length (mm)')
plt.ylabel('Bill Depth (mm)')

# Mostra il grafico
plt.show()

In [ ]:
import numpy as np

# Informazioni sul dataset
print("\nInformazioni sul dataset:")
print(df_senza_mancanti.info())

# Statistiche descrittive
print("\nStatistiche descrittive:")
print(df_senza_mancanti.describe())




In [ ]:

# Distribuzione delle classi nel target
plt.figure(figsize=(6, 4))
df_senza_mancanti['species'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Distribuzione delle classi nel target')
plt.xlabel('Classe')
plt.ylabel('Conteggio')
plt.xticks(rotation=0)
plt.show()


# Divisione in train e test set

La divisione del dataset in un training set e un test set è un passo fondamentale nell'addestramento e nella valutazione di modelli di apprendimento automatico. Questa pratica aiuta a valutare l'efficacia del modello su dati non visti, fornendo una stima delle sue prestazioni su nuovi dati.

In [ ]:
# Otteniamo un subset X senza la nostra variabile target
X = df_senza_mancanti.drop(['species','island','sex'],axis=1)
X.head()


In [ ]:
# Andiamo a selezionare la variabile che vogliamo classificare etichettandola come y
y = df_senza_mancanti['species']
y.head()

In [ ]:
from sklearn.model_selection import train_test_split


# Suddividere il dataset in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Dopo la suddivisione del dataset in training e test set, è importante esaminare la forma (shape) e la distribuzione dei dati in ciascun set. Ciò ti fornirà una visione dettagliata della dimensione dei set e dell'equilibrio delle classi. Di seguito, ti mostro come ottenere queste informazioni utilizzando Python e la libreria pandas.

In [ ]:
# Converti array NumPy in oggetti Series di pandas
y_train_series = pd.Series(y_train)
y_test_series = pd.Series(y_test)

# Informazioni sulla forma (shape) dei set di addestramento e test
print("Shape del set di addestramento (X_train, y_train):", X_train.shape, y_train_series.shape)
print("Shape del set di test (X_test, y_test):", X_test.shape, y_test_series.shape)

# Distribuzione delle classi nei set di addestramento e test
print("\nDistribuzione delle classi nel set di addestramento:")
print(y_train_series.value_counts(normalize=True))

print("\nDistribuzione delle classi nel set di test:")
print(y_test_series.value_counts(normalize=True))


# Applicazione algoritmo

In [ ]:
# Importa il modulo dell'albero decisionale da scikit-learn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import tree

# Crea un'istanza del modello di albero decisionale
tree_model = DecisionTreeClassifier(random_state=42)

# Addestra il modello sull'insieme di addestramento
tree_model.fit(X_train, y_train)

# Fai predizioni sull'insieme di test
y_pred = tree_model.predict(X_test)

# Valuta le prestazioni del modello
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy del modello: {accuracy:.2f}')

# Visualizza la matrice di confusione
conf_matrix = confusion_matrix(y_test, y_pred)
print('\nMatrice di Confusione:')
print(conf_matrix)

# Visualizza il report di classificazione
class_report = classification_report(y_test, y_pred)
print('\nReport di Classificazione:')
print(class_report)

# Visualizza l'albero decisionale
plt.figure(figsize=(15, 10))
tree.plot_tree(tree_model, feature_names=X.columns, class_names=['Adelie', 'Gentoo','Chinstrap'], filled=True, rounded=True)
plt.show()


Le metriche di qualità sono fondamentali per valutare le prestazioni di un modello di classificazione.

In [ ]:
# Calcola e stampa le metriche di qualità del modello
precision = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[0, 1])
recall = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1_score:.2f}')

1. **Precision (Precisione):**
   - La precisione misura la percentuale di istanze positive predette dal modello che sono realmente positive.
   - Formula: `precision = TP / (TP + FP)`, dove TP è il numero di True Positives e FP è il numero di False Positives.

2. **Recall (Recupero o Sensibilità):**
   - Il recall misura la percentuale di istanze positive reali che sono state predette correttamente dal modello.
   - Formula: `recall = TP / (TP + FN)`, dove TP è il numero di True Positives e FN è il numero di False Negatives.

3. **F1 Score:**
   - L'F1 Score è la media armonica di precision e recall, fornendo una singola metrica che tiene conto di entrambi.
   - Formula: `F1 Score = 2 * (precision * recall) / (precision + recall)`.

**Interpretazione:**
- Una precisione elevata indica che quando il modello predice una classe positiva, è molto probabile che sia corretto.
- Un recall elevato indica che il modello è in grado di individuare la maggior parte delle istanze positive reali.
- L'F1 Score è particolarmente utile quando precisione e recall devono essere bilanciati.

Ricorda che le metriche possono variare a seconda del contesto e degli obiettivi specifici del problema di classificazione. Ad esempio, in un contesto medico, potrebbe essere più critico avere un recall elevato per garantire la rilevazione di tutti i casi positivi.

L'uso di queste metriche ti aiuterà a ottenere una comprensione più approfondita delle prestazioni del tuo modello e a prendere decisioni informate sulla sua efficacia nella risoluzione del problema di classificazione.